In [1]:
%matplotlib inline
import pandas as pd
import numpy as np

import json
import os
import openpyxl as opxl
from itertools import islice

## Funciones

In [2]:
def load_from_file(route, file_format, columns, medidas):
    files = os.listdir(route)
    for file_name in files:
        if '.xlsx' in file_name:
            wb = opxl.load_workbook(filename = file_name, data_only=True)
            ws = wb['Data']

            experiment_name = os.path.basename(file_name).split('/')[-1].split('.')[0]

            if file_format == 'bmg':
                machine_name = 'bmg'
                df_cons = load_bmg_data(ws, columns)
            elif file_format == 'synergy':
                machine_name = ws['B'][8].value + str(ws['B'][9].value)
                df_cons = load_synergy_data(medidas, ws)
            #dict_meta, e = load_meta_info(wb, experiment_name, machine_name)           
            dict_meta = load_meta_info(wb, experiment_name, machine_name)
            
            #upload_data(e, dict_meta, df_cons, columns)
            upload_data(dict_meta, df_cons, columns)
        else:
            # Get an error message
            pass

In [3]:
# NOT USING IT

def load(file_name, file_format, experiment_name=None, machine_name=None):
    # Needs to get all experiments:
    existing_exps = []
    if not experiment_name:
        experiment_name = os.path.basename(file_name)
    
    # If experiment already exists
    if experiment_name in existing_exps:
            print('Experiment ', experiment_name, ' already exists in database. Have you already uploaded this file, or one with the same name? Skipping file.')
    else:
        wb = opxl.load_workbook(filename = file_name, data_only=True)
        ws = wb['Data']
        print('Uploading data from file '+file_name)
        
        if 'synergy' in file_format:
            if not machine_name:
                machine_name = st['B'][8].value + str(st['B'][9].value)
            exp = load_meta_info(wb, experiment_name, machine_name, session, engine)
            #exp = load_synergy_data(st, exp, session, medidas)
            pass
        elif 'bmg' in file_format:
            if not machine_name:
                machine_name = 'bmg'
            exp = load_meta_info(wb, experiment_name, machine_name, session, engine)
            exp = load_bmg_data(st, exp, session, engine)
            pass
        else:
            print('File format %s not supported'%file_format)

In [4]:
def load_meta_info(wb, experiment_name, machine_name):
    # e = Experiment(name=experiment_name, machine=machine_name)
    # e.save()
    
    # Dictionary where dataframes will be stored and send
    meta_dict = {'Strain_name':'', 'Strain_df':'', 
                 'Media_name':'', 'Media_df':'', 
                 'DNA_name':'', 'DNA_df':'', 
                 'Inducer_name':'', 'Inducer_df':''}

    meta_dict['Strain_name'], meta_dict['Strain_df'] = table_values(wb['Strains'], 1)
    meta_dict['Media_name'], meta_dict['Media_df'] = table_values(wb['Media'], 1)
    meta_dict['DNA_name'], meta_dict['DNA_df'] =  get_all_tables(wb['DNA'])
    
    if 'Inducers' in wb.sheetnames:
        meta_dict['Inducer_name'], meta_dict['Inducer_df'] = get_all_tables(wb['Inducers'])
    else:
        meta_dict['Inducer_name'], meta_dict['Inducer_df'] = [], []
    
    # It also should return the experiment created
    return meta_dict#, e

In [5]:
def upload_data(e, dict_meta, df_cons, columns):
    # El método load_meta_info retorna el experimento e
    for index, value in enumerate(columns):
        row = value[0]
        col = int(value[1:])
        # existing_dna = [i.name for i in Dna.objects.all()]
        existing_dna = []
        # Metadata value for each well
        s_strain = dict_meta['Strain_df'].at[row, col]
        s_media = dict_meta['Media_df'].at[row, col]
        # s = Sample(experiment=e, row=row, col=col, media=s_media, strain=s_strain)
        # s.save()

        for df_dna in dict_meta['DNA_df']:
            s_dna = df_dna.at[row, col]
            if s_dna != 'None':
                if s_dna not in existing_dna:
                    # d = Dna(name=s_dna, sboluri='')
                    # d.save()
                    # v = Vector(dna=d, sample=s)
                    # v.save()
                    pass
                else:                    
                    # d = Dna.objects.filter(name__exact=s_dna)[0]
                    # v = Vector(dna=d, sample=s)
                    # v.save()
                    pass

        if len(dict_meta['Inducer_df']) > 0:
            for i, df_ind in enumerate(dict_meta['Inducer_df']):
                conc = df_ind.at[row, col]
                ind_name = dict_meta['Inducer_name'][i]
                # i = Inducer(sample=s, concentration=conc, pubchemid=ind_name)
                # i.save()

        # Data value for earch well
        for i, val in enumerate(df_cons[row+str(col)]):
            m_name = df_cons['name'].iloc[i]
            m_value = val
            m_time = df_cons['Time'].iloc[i]
            # m = Measurement(sample=s, name=m_name, value=m_value, time=m_time)
            # m.save()

In [6]:
def find_in_sublists(lst, value):
    for sub_i, sublist in enumerate(lst):
        try:
            return (sub_i, sublist.index(value))
        except ValueError:
            pass
    raise ValueError('%s is not in lists' % value)

In [7]:
def get_all_tables(ws):
    # Return a list of tables (8x12) as lists in worksheet
    # Tables are assumed to be ordered vertically
    length = len(ws['A'])
    ntables = (length + 1) // 10
    table_list = []
    names_list = []
    for i in range(ntables):
        name,values = table_values(ws, i*10+1)
        table_list.append(values)
        names_list.append(name)
    return names_list,table_list

In [8]:
# It returns a 96 items list, from west to east, north to south from the plate. 
# Now it returns a df
def table_values(ws, row):
    name = ws[row][0].value
    vals = []
    for cell_row in range(row + 1, row + 9):
        for cell_col in range (1, 13):
            val = ws[cell_row][cell_col].value
            if not val:
                vals.append(0)
            else:
                vals.append(val)
    df_vals = pd.DataFrame(np.reshape(np.array(vals), (8,12)), 
                               index=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'], 
                               columns=range(1,13))
    return name, df_vals

In [9]:
def clean_bmg_data(times, rows_data, names, columns):
    l = int(len(rows_data[0])/3)
    
    t_CFP = times[:l]
    CFP = [r[:l] for r in rows_data]
    t_YFP = times[l:160]
    YFP = [r[l:160] for r in rows_data]
    t_OD = times[160:]
    OD = [r[160:] for r in rows_data]

    list_data = [CFP, YFP, OD]
    list_time = [t_CFP, t_YFP, t_OD]
    
    # List with time series for 3 measurements
    cols = columns[:]
    cols.insert(0, 'Time')
    cols.append('name')

    df_cons = pd.DataFrame(columns= cols) 
    for i, value in enumerate(list_data):
        df = pd.DataFrame(value)
        df = df.transpose()
        df.columns = columns
        df.insert(0, 'Time', list_time[i])
        df['name'] = names[i]
        df_cons = df_cons.append(df, ignore_index=True)
    return df_cons

In [10]:
def load_bmg_data(ws, columns):
    rawdata = [[cell.value for cell in row] for row in ws.iter_rows()]

    # Work out location of header row, and column/row numbers
    try:
        (headerrow,ccol) = find_in_sublists(rawdata,'Well Col')
        (headerrow,crow) = find_in_sublists(rawdata,'Well Row')
    except ValueError:
        (headerrow,ccol) = find_in_sublists(rawdata,'Well\nCol')
        (headerrow,crow) = find_in_sublists(rawdata,'Well\nRow')

    # Time is on row below headers
    timerow = headerrow+1
    # Data is on rows below time
    datarow = timerow+1
    headers = rawdata[headerrow]

    # Find where the data is in each row by looking for 1st 'Raw Data'
    cdata = next((i for i,v in enumerate(headers) if 'Raw Data' in v))

    # Pull out row,col and data for each row
    # Try combinations of alpha/numeric for row/col
    try:
        rows = [(ord(row[crow].upper())-64, int(row[ccol]), [float(v) for v in row[cdata:]]) for row in rawdata[datarow:]]
    except ValueError:
        rows = [(int(row[ccol]), ord(row[crow].upper())-64, [float(v) for v in row[cdata:]]) for row in rawdata[datarow:]]

    # Pull out time of each data point
    times = rawdata[timerow][cdata:]
    # Put just data apart
    rows_data = [r[2] for r in rows]
    names = ['CFP', 'YFP', 'OD']

    df_cons = clean_bmg_data(times, rows_data, names, columns)
    
    return df_cons

In [11]:
# Cleans the main df
def clean_synergy_data(names, df, lista_rows):
    df_cons = pd.DataFrame(columns=df.columns)
    for i, value in enumerate(lista_rows):
        if i == 0:
            df2 = pd.DataFrame(df.iloc[0:lista_rows[i][1] - 3])
        else:
            df2 = pd.DataFrame(df.iloc[lista_rows[i-1][1] + 1:lista_rows[i][1] - 3])
        df2['name'] = names[i]
        fix_synergy_time(df2)
        df_cons = df_cons.append(df2, ignore_index=True, sort=False)
    return df_cons

In [12]:
# Changes time's format from datetime to fraction
def fix_synergy_time(df):
    t = np.array([])
    for i, value in enumerate(df['Time']):
        if i > 0:
            if df['Time'].iloc[i].hour < df['Time'].iloc[i-1].hour:
                t = np.append(t, [24 + value.hour + value.minute/60 + value.second/3600])
            else:
                t = np.append(t, [value.hour + value.minute/60 + value.second/3600])
        else:
            t = np.append(t, [value.hour + value.minute/60 + value.second/3600])
    df['Time'] = t

In [13]:
def lista_rows(ws, medidas):
    lista_rows = [(celda.value, celda.row, opxl.utils.column_index_from_string(celda.column)) 
                  for celda in ws['A'] 
                  if celda.value in medidas]
    return lista_rows

In [14]:
def load_synergy_data(medidas, ws):
    rows_ini = lista_rows(ws, medidas)
    ws.delete_rows(0, rows_ini[0][1] + 1)
    rows = lista_rows(ws, medidas)

    data = ws.values
    cols = next(data)[1:]
    data = list(data)
    idx = [r[0] for r in data]
    data = (islice(r, 1, None) for r in data)
    df = pd.DataFrame(data, columns=cols)
    df = df.drop('T° OD600:600', axis=1)
    
    name_map = {'OD600:600':'OD', 'RFP-YFP:500/27,540/25':'YFP', 'CFP:420/50,485/20':'CFP', 'RFP-YFP:585/10,620/15':'RFP'}
    names = [name_map[rows_ini[i][0]] for i in range(len(rows_ini)-1)]
    
    df_cons = clean_synergy_data(names, df, rows)
    return df_cons

## Carga de Archivos 

In [ ]:
columns = [x+str(y) for x in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'] for y in range(1,13)]
medidas = ['OD600:600', 'RFP-YFP:585/10,620/15', 'RFP-YFP:500/27,540/25', 'CFP:420/50,485/20', 'Results']


load_from_file('datafiles/bmg/to_upload/', 'bmg', columns, medidas)
load_from_file('datafiles/synergy/to_upload/', 'synergy', columns, medidas)

## Testeo de carga

### BMG

In [15]:
# Tirar este desde un main para que lo haga con todos los archivos de la carpeta

columns = [x+str(y) for x in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'] for y in range(1,13)]

files = os.listdir('datafiles/bmg/to_upload/')
file_name = 'datafiles/bmg/to_upload/Tim260913.xlsx'
experiment_name = os.path.basename(file_name).split('/')[-1].split('.')[0]
machine_name = 'M1'
wb = opxl.load_workbook(filename = file_name, data_only=True)
ws = wb['Data']

/home/guillermo/anaconda3/lib/python3.6/site-packages/openpyxl/reader/worksheet.py:318: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [16]:
dict_meta = load_meta_info(wb, experiment_name, machine_name)
df_cons = load_bmg_data(ws, columns)

In [17]:
df_cons

,Time,A1,A2,A3,A4,A5,A6,A7,A8,A9,...,H4,H5,H6,H7,H8,H9,H10,H11,H12,name
0,0.000000,4058.0000,4132.0000,4200.0000,4170.0000,4179.0000,4206.0000,4197.0000,4060.0000,4128.0000,...,4068.0000,4052.0000,4086.0000,4095.0000,4082.0000,4108.0000,4051.0000,4131.0000,4098.0000,CFP
1,0.196389,3927.0000,3988.0000,4044.0000,4021.0000,3975.0000,4116.0000,4059.0000,4031.0000,4076.0000,...,3972.0000,3898.0000,3931.0000,3940.0000,3938.0000,3947.0000,3920.0000,3930.0000,3925.0000,CFP
2,0.393056,3902.0000,4007.0000,4008.0000,4045.0000,4005.0000,4017.0000,4026.0000,3972.0000,3997.0000,...,3960.0000,3920.0000,3951.0000,3904.0000,3941.0000,3984.0000,3902.0000,3855.0000,3940.0000,CFP
3,0.590556,3937.0000,3897.0000,3977.0000,4035.0000,3987.0000,4050.0000,3985.0000,3958.0000,3960.0000,...,3961.0000,3887.0000,3954.0000,3951.0000,3848.0000,3891.0000,3872.0000,3942.0000,3996.0000,CFP
4,0.787222,3950.0000,4005.0000,4122.0000,3984.0000,3991.0000,4003.0000,3919.0000,3932.0000,4033.0000,...,3924.0000,3889.0000,3984.0000,3957.0000,3979.0000,3892.0000,3818.0000,3813.0000,3999.0000,CFP
5,0.984722,3878.0000,4012.0000,4020.0000,3928.0000,3991.0000,3928.0000,3968.0000,3911.0000,3934.0000,...,3917.0000,3891.0000,4073.0000,3952.0000,3857.0000,3938.0000,3878.0000,3918.0000,3975.0000,CFP
6,1.182222,3882.0000,3909.0000,3964.0000,3966.0000,3954.0000,3982.0000,3977.0000,3998.0000,3921.0000,...,3958.0000,3922.0000,3971.0000,3986.0000,3898.0000,3900.0000,3824.0000,3898.0000,3905.0000,CFP
7,1.378889,3957.0000,3916.0000,3983.0000,4009.0000,3956.0000,3961.0000,3992.0000,3956.0000,3932.0000,...,3955.0000,3947.0000,3904.0000,3917.0000,3900.0000,3888.0000,3894.0000,3910.0000,3994.0000,CFP
8,1.576667,4016.0000,3952.0000,4121.0000,4025.0000,4022.0000,4176.0000,4015.0000,3972.0000,4083.0000,...,3992.0000,3880.0000,4016.0000,3945.0000,3896.0000,3952.0000,3756.0000,3917.0000,3950.0000,CFP
9,1.774167,4002.0000,3984.0000,4047.0000,4040.0000,4065.0000,4043.0000,3984.0000,3989.0000,4023.0000,...,3982.0000,3980.0000,3987.0000,4023.0000,3954.0000,3940.0000,3932.0000,3925.0000,3927.0000,CFP


### Synergy

In [28]:
# Esta info debe ser recibida al inicio de la carga de datos
file_name = 'datafiles/synergy/to_upload/Cinetica 1 rep 3.xlsx'
experiment_name = os.path.basename(file_name).split('/')[-1].split('.')[0]

wb = opxl.load_workbook(filename = file_name, data_only=True)
ws = wb['Data']
machine_name = ws['B'][8].value + str(ws['B'][9].value)

#Debería venir desde el front
medidas = ['OD600:600', 'RFP-YFP:585/10,620/15', 'RFP-YFP:500/27,540/25', 'CFP:420/50,485/20', 'Results']

In [29]:
dict_meta = load_meta_info(wb, experiment_name, machine_name)
df_cons = load_synergy_data(medidas, ws)

In [ ]:
Test
Debería subir
- synergy: 9312*4*3 = 111744
- bmg: 23040*3 = 69120
TOTAL: 180864

In [ ]:
Real
Debería subir
- synergy: 9312*4*12 = 446976
- bmg: 23040*3 = 69120
TOTAL: 516096